In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [26]:
df = spark.read.option("header", True).csv('./Baby_Names__Beginning_2007.csv')
df

DataFrame[Year: string, First Name: string, County: string, Sex: string, Count: string]

In [27]:
df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Count: string (nullable = true)



In [28]:
df.show()

+----+----------+------+---+-----+
|Year|First Name|County|Sex|Count|
+----+----------+------+---+-----+
|2021|    OLIVIA|Albany|  F|   16|
|2021|    AMELIA|Albany|  F|   16|
|2021|    SOPHIA|Albany|  F|   13|
|2021|       AVA|Albany|  F|   13|
|2021|      EMMA|Albany|  F|   12|
|2021|  PENELOPE|Albany|  F|   11|
|2021|   MADISON|Albany|  F|   10|
|2021|   ABIGAIL|Albany|  F|   10|
|2021|    HARPER|Albany|  F|   10|
|2021|      ISLA|Albany|  F|    9|
|2021| CHARLOTTE|Albany|  F|    8|
|2021|  ISABELLA|Albany|  F|    8|
|2021|    GIANNA|Albany|  F|    8|
|2021|   ELEANOR|Albany|  F|    8|
|2021|   LILLIAN|Albany|  F|    8|
|2021|  MADELINE|Albany|  F|    8|
|2021|    EVELYN|Albany|  F|    7|
|2021|     RILEY|Albany|  F|    7|
|2021|    HAILEY|Albany|  F|    7|
|2021|    EVERLY|Albany|  F|    7|
+----+----------+------+---+-----+
only showing top 20 rows



In [32]:
normalized_columns_names = [col.lower().replace(' ', '_') for col in df.columns]
normalized_columns_names

['year', 'first_name', 'county', 'sex', 'count']

In [34]:
df = df.toDF(*normalized_columns_names)
df.show()

+----+----------+------+---+-----+
|year|first_name|county|sex|count|
+----+----------+------+---+-----+
|2021|    OLIVIA|Albany|  F|   16|
|2021|    AMELIA|Albany|  F|   16|
|2021|    SOPHIA|Albany|  F|   13|
|2021|       AVA|Albany|  F|   13|
|2021|      EMMA|Albany|  F|   12|
|2021|  PENELOPE|Albany|  F|   11|
|2021|   MADISON|Albany|  F|   10|
|2021|   ABIGAIL|Albany|  F|   10|
|2021|    HARPER|Albany|  F|   10|
|2021|      ISLA|Albany|  F|    9|
|2021| CHARLOTTE|Albany|  F|    8|
|2021|  ISABELLA|Albany|  F|    8|
|2021|    GIANNA|Albany|  F|    8|
|2021|   ELEANOR|Albany|  F|    8|
|2021|   LILLIAN|Albany|  F|    8|
|2021|  MADELINE|Albany|  F|    8|
|2021|    EVELYN|Albany|  F|    7|
|2021|     RILEY|Albany|  F|    7|
|2021|    HAILEY|Albany|  F|    7|
|2021|    EVERLY|Albany|  F|    7|
+----+----------+------+---+-----+
only showing top 20 rows

